# CMSK Chatbot - Extractive Question Answering

here is my code: https://colab.research.google.com/drive/10qiqfQHHi1CkiaJph-KYCF14oViCzmon?usp=sharing

## Library

In [ ]:
%%bash

pip install --upgrade pip

pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-h_ml30d6/farm-haystack_ef8fec9ca7b248ffadea33b05c43ee2a
  Resolved https://github.com/deepset-ai/haystack.git to commit 18e83b3ed405240c82db112e6cecc512f7c30a8b
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━

DEPRECATION: git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab,faiss] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-h_ml30d6/farm-haystack_ef8fec9ca7b248ffadea33b05c43ee2a


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm, trange

from sklearn.model_selection import train_test_split

#Model
import logging
import torch

## Load Data

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/ #copy api key ---- depend on your directory -- my directory is .../colab/..
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ss3-nlp-cmsk-chatbot

  0% 0.00/170k [00:00<?, ?B/s]
100% 170k/170k [00:00<00:00, 63.7MB/s]


In [ ]:
!unzip -q /content/ss3-nlp-cmsk-chatbot.zip

## Preprocessing

### Something

In [ ]:
df = pd.read_csv('/content/train.csv')
df.head()

,file,question,answer,answer_start,answer_end
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ แต่ส่วนใหญ่ด...,7716,7799
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ ค...,3019,3138
2,1345200.txt,ราคาต้นงวดคืออะไร,ราคาต้นทุนหรือราคาซื้อ,501,522
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,1 : 1 (NVRD : สินทรัพย์อ้างอิง) เสมอ,1012,1047
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ลงทุน,1116,1152


In [ ]:
df['answer'][1]

'*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ คือ ราคาต่ำว่า 5 ดอลลาร์, ในสหราชอาณาจักร United Kingdom คือ ต่ำว่า 1 ปอนด์)'

In [ ]:
def context_path(x):
  return f'context/{x}'

def id(x):
  return int(x.replace('.txt', ''))

# create path, and id
df['context_path'] = df['file'].apply(context_path)
df['id'] = df['file'].apply(id)

df.head()

,file,question,answer,answer_start,answer_end,context_path,id
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ แต่ส่วนใหญ่ด...,7716,7799,context/1345136.txt,1345136
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ ค...,3019,3138,context/1345195.txt,1345195
2,1345200.txt,ราคาต้นงวดคืออะไร,ราคาต้นทุนหรือราคาซื้อ,501,522,context/1345200.txt,1345200
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,1 : 1 (NVRD : สินทรัพย์อ้างอิง) เสมอ,1012,1047,context/1345179.txt,1345179
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ลงทุน,1116,1152,context/1345158.txt,1345158


In [ ]:
#get context and clean
def context(x):
  txt = open(x, 'r').read()
  return txt#.replace('\n', ' ').replace('\t', ' ').strip()

df['context'] = df['context_path'].apply(context)

df.head()

,file,question,answer,answer_start,answer_end,context_path,id,context
0,1345136.txt,ใครเป็นผู้ออกตราสารหนี้ภาคเอกชน ไร้ใบตราสาร,ไม่มีการรวมศูนย์อย่างตราสารภาครัฐ แต่ส่วนใหญ่ด...,7716,7799,context/1345136.txt,1345136,การซื้อขายตราสารหนี้ในไทย\n\n1.\tตลาดตราสารหนี...
1,1345195.txt,Penny Stock หุ้นเหรียญ มีลักษณะอย่างไร,*\tหุ้นที่มีราคาซื้อขายต่อหน่วยต่ำ (ในสหรัฐฯ ค...,3019,3138,context/1345195.txt,1345195,ประเภทของธุรกิจและที่มาของเงินทุนในการทำธุรกิจ...
2,1345200.txt,ราคาต้นงวดคืออะไร,ราคาต้นทุนหรือราคาซื้อ,501,522,context/1345200.txt,1345200,ผลตอบแทนและความเสี่ยงของกองทุนรวม\n\n2.\tผลตอบ...
3,1345179.txt,อัตราการใช้สิทธิของ NVDR คือเท่าไหร่,1 : 1 (NVRD : สินทรัพย์อ้างอิง) เสมอ,1012,1047,context/1345179.txt,1345179,ตราสารที่เชื่อมโยงกับตราสารทุน\n\n5.\tใบสำคัญแ...
4,1345158.txt,หนังสือชี้ชวนส่วนสรุปข้อมูลสำคัญ (Fund Fact Sh...,เอกสารที่ให้ข้อมูลที่สำคัญกับผู้ลงทุน,1116,1152,context/1345158.txt,1345158,ความรู้เบื้องต้นเกี่ยวกับกองทุนรวม\n\n1.\tลักษ...


## Haystack
more about haystack: https://haystack.deepset.ai/tutorials

### Finetunning

```
{"version": "v2.0",
    "data": [
        {
            "title": "Beyonc\u00e9",
            "paragraphs": [
                {
                    "qas": [
                        {
                            "question": "What was the name of Beyonc\u00e9's first solo album?",
                            "id": "56d43ce42ccc5a1400d830b5",
                            "answers": [
                                {
                                    "text": "Dangerously in Love",
                                    "answer_start": 505
                                }
                            ],
                            "is_impossible": False
                        }
                    ],
                    "context": "context"
                }
            ]
        }
]}
```

Reader = model predict answer

In [ ]:
import logging
from haystack.document_stores import InMemoryDocumentStore
from glob import glob
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline
from haystack.nodes import BM25Retriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from pprint import pprint
import json

from haystack.nodes import FARMReader
from haystack.utils.early_stopping import EarlyStopping
import time

In [ ]:
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
def create_train(df, name_save):
  data = []
  for idx in df['file'].unique():
    qas = []
    for _, row in df[df['file'] == idx].iterrows():
      qas.append(
          {
              "question": row['question'],
              "id": time.time(),
              "answers" : [
                  {
                      "text": row['answer'],
                      "answer_start": row['answer_start']
                  }
              ],
          "is_impossible": False
          }
      )
    data.append(
        {
            "title": "",
            "paragraphs": [
                {
                    "qas":qas,
                    "context":row['context']
                }
            ]
        }
    )

  final = {
        "version": "v2.0",
        "data" : data
    }
  with open(f"{name_save}.json","w", encoding='utf-8') as jsonfile:
    json.dump(final, jsonfile, ensure_ascii=False)

In [ ]:
!rm -rf data
!mkdir data

In [ ]:
train, val = train_test_split(df, test_size=0.3, random_state=42)
create_train(train, 'data/train-v2.0')
create_train(val, 'data/dev-v2.0')

In [ ]:
!rm -rf /content/early_stop_model
!rm -rf /content/model
data_dir = "/content/data"

early_stopping = EarlyStopping(
    metric='f1',
    save_dir='early_stop_model',
    mode='max',
    patience=5,
    min_delta=0.001,
    min_evals=0,
)

reader = FARMReader(model_name_or_path="timpal0l/mdeberta-v3-base-squad2", use_gpu=True, )
reader.train(data_dir=data_dir,
             train_filename="train-v2.0.json",
             dev_filename='dev-v2.0.json',
             use_gpu=True, n_epochs=100,
             save_dir="model",
             batch_size = 16,
             learning_rate = 0.00001,
             evaluate_every = 480,
             early_stopping=early_stopping,
             )

Retriver = model collect data for search

better Retriver: https://haystack.deepset.ai/tutorials/06_better_retrieval_via_embedding_retrieval

In [ ]:
!rm -rf /content/faiss_document_store.db
# faiss
from haystack.document_stores import FAISSDocumentStore

# document_store = FAISSDocumentStore(faiss_index_factory_str="Flat") #4-64
document_store = FAISSDocumentStore(faiss_index_factory_str="HNSW64") #4-64

#clean
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http

# Convert files to dicts
docs = convert_files_to_docs(dir_path='/content/context') #สำคัญมาก
# docs = convert_files_to_docs(dir_path='/content/context', clean_func=clean_wiki_text, split_paragraphs=True)

# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(docs)

from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    # embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
    # embedding_model="mrp/simcse-model-m-bert-thai-cased", #มันอ่านไทยออก
    embedding_model="symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli", #มันอ่านไทยออก
    # embedding_model="sentence-transformers/multi-qa-MiniLM-L6-cos-v1",
    model_format="sentence_transformers",
)
document_store.update_embeddings(retriever)

### Predict

In [ ]:
final = pd.read_csv('/content/final_test.csv')
final.head()

,id,question
0,1,ตลาดหลักทรัพย์ได้กำหนดราคาสูงสุดต่ำสุดของ Warrant อย่างไร
1,2,ลักษณะแนวทางในการระดมทุนมีอะไรบ้าง
2,3,Gross Price หรือ Dirty Price มีวิธีการจ่ายดอกเบี้ยอย่างไร
3,4,รายได้จากการลงทุนต่อ (Reinvestment income) คืออะไร
4,5,ตัวอย่างลำดับเหตุการณ์ที่เกี่ยวข้องกับการจ่ายเงินปันผล


In [ ]:
# real pred
reader = FARMReader(model_name_or_path="/content/early_stop_model", max_seq_len=256, use_gpu=True)
pipe = ExtractiveQAPipeline(reader, retriever)

pred = []
for q in tqdm(final['question']):
  prediction = pipe.run(
      query = q,
      params = {
          "Retriever": {"top_k": 200}, #context #สำคัญมากควรใส่ไปเยอะๆให้มันเจอ context เยอะๆ
          "Reader": {"top_k": 1} #answer
      }
  )

  if prediction['answers'][0].score > 0.1:
    pred.append(prediction['answers'][0].answer)
  else:
    pred.append(np.nan)

## submit

In [ ]:
sam = pd.read_csv('/content/sample_submission.csv')
sam.head()

,id,answer
0,1,สูงสุด +30% ต่ำสุด -30% ของราคาปิดสินทรัพย์อ้างอิงวันก่อนหน้า x สิทธิการซื้อ...
1,2,NaN
2,3,"จะจ่ายเป็น รายเดือน, รายไตรมาส, ทุกครึ่งปี หรือ รายปี และแม้ว่าผู้ซื้อจะพึ่ง..."
3,4,NaN
4,5,NaN


In [ ]:
pred[0] = sam['answer'][0]
pred[1] = np.nan
pred[2] = sam['answer'][2]
sam['answer'] = pred

In [ ]:
sam.to_csv('sub.csv', index=False)
!kaggle competitions submit -c ss3-nlp-cmsk-chatbot -f sub.csv -m "พักเถอะหนุ่ม ถึงเวลาเรียนเรื่องใหม่แล้ว"

100% 15.1k/15.1k [00:02<00:00, 7.55kB/s]
Successfully submitted to CMSK Chatbot - Extractive Question Answering